Running Your Queries In Spark
---------------------------

You need to take the data from Foursquare and perform your analysis based on the question you chose.

In our example below, we do the following:
1. We read the files that our Foursquare client generates from the drive.
2. For each city, we get the trending venue categories and the number of people currently being there.
3. We add up the numbers for the same categories.

You can extend this into a web dashboard, or plots inside this notebook if you choose.

----------------------------------------------------------------------------------------------------------------------

In [1]:
# Install Java, Spark, Findspark and PySpark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

!pip install -q findspark
!pip install pyspark

# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

    100% |████████████████████████████████| 213.4MB 93kB/s 
    100% |████████████████████████████████| 204kB 32.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/cd/54/c2/abfcc942eddeaa7101228ebd6127a30dbdf903c72db4235b23
Successfully built pyspark
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


#### Import the relevant modules

In [0]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession

#### The code below deletes all the log files inside the foursquare_logs directory

In [0]:
import shutil
folder = "/content/gdrive/My Drive/Colab Datasets/foursquare_logs"
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

#### We now create a helper function that allow us to store the aggregate number for each category.

In [0]:
def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)

In [0]:
# create spark configuration
conf = SparkConf()
conf.setAppName("FoursquareStreamApp")

# create spark context with the above configuration
sc = SparkContext.getOrCreate(conf=conf)
sc.setLogLevel("ERROR")

# create the Streaming Context from the above spark context with 
# interval size 10 seconds
ssc = StreamingContext(sc,10)

# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_FoursquareApp")

# read data from drive
dataStream = ssc.textFileStream("/content/gdrive/My Drive/Colab Datasets/foursquare_logs")

#### Finally, we implement our primary workflow.

After the implementation of our workflow, we begin the streaming with `ssc.start()`. The query stays open until we terminate it (`ssc.awaitTermination()`).

In [6]:
visitor_counts = dataStream.map(lambda x: (x.split(",")[0], int(x.split(",")[1]))).reduceByKey(lambda a, b: a + b)

runningCounts = visitor_counts.updateStateByKey(updateFunction)

runningCounts.pprint()

# start the streaming computation
ssc.start()

# wait for the streaming to finish
ssc.awaitTermination()

-------------------------------------------
Time: 2019-02-10 21:22:30
-------------------------------------------

-------------------------------------------
Time: 2019-02-10 21:22:40
-------------------------------------------

-------------------------------------------
Time: 2019-02-10 21:22:50
-------------------------------------------
('Art Museum', 15)
('Train Station', 10)
('Coffee Shop', 10)
('Gourmet Shop', 6)
('Market', 8)
('Multiplex', 8)
('Theater', 7)
('Park', 6)
('Exhibit', 6)

-------------------------------------------
Time: 2019-02-10 21:23:00
-------------------------------------------
('Art Museum', 15)
('Train Station', 10)
('Coffee Shop', 16)
('Gourmet Shop', 6)
('Café', 9)
('American Restaurant', 6)
('Brewery', 7)
('Market', 8)
('Multiplex', 8)
('Theater', 7)
...

-------------------------------------------
Time: 2019-02-10 21:23:10
-------------------------------------------
('Art Museum', 15)
('Train Station', 10)
('Coffee Shop', 16)
('Gourmet Shop', 6)
('Café

KeyboardInterrupt: ignored

#### When the running process halts, you may need to stop the current Spark Context by running the following cell:

In [0]:
ssc.stop()